In [10]:
using PyCall

In [30]:
function gen_instances(t, p, fp = false)
    if isfile(fp)
        
    else
        T = 1:t
        P = 1:p
        alpha = round(0.6 + (0.95 - 0.6) * rand(), digits = 2)
        set_up_cost = [rand(500:1000) for _ in P, _ in T]
        variable_prod_cost = [rand(10:14) for _ in P, _ in T]
        holding_cost = [rand(5:10) for _ in P, _ in T]
        demand = [rand(0:50) for _ in P, _ in T]
        mtn_cost = [rand(1000 +3*p*t:5000 + 3*p*t) for _ in T]
        cmax = p*rand(40:50)

LoadError: UndefVarError: path not defined

In [ ]:
function mutation(sol, instance_dict, lag_step)
    T = instance_dict["T"]
    P = instance_dict["P"]
    cmax = instance_dict["cmax"]
    alpha = instance_dict["alpha"]
    t = length(T)
    demand = instance_dict["demand"]
    mtn_cost = instance_dict["mtn_cost"]
    set_up_cost = instance_dict["set_up_cost"]
    variable_prod_cost = instance_dict["variable_prod_cost"]
    holding_cost = instance_dict["holding_cost"]
    unf_cost = sum(variable_prod_cost[i,j]*demand[i,j] + holding_cost[i,j]*demand[i,j] + set_up_cost[i,j] for i in P, j in T) + sum(mtn_cost[j] for j in T)
    #ind = rand(2:t)
    z_new = copy(sol.z)
    c_new = copy(sol.c)
    y_new = copy(sol.y)
    
    #println(z_new)
    #display(y_new)
    list_ind = sort(randperm(t-1) .+ 1)
    
    i = 1
    while i < t && z_new[list_ind[i]] != 1 && list_ind[i] + lag_step < t
        i += 1
    end
    #println(list_ind[i])
    #println(z_new[list_ind[i]])
    if z_new[list_ind[i] + lag_step] == 0 
        z_new[list_ind[i]] = 0 
        c_new[list_ind[i]] = alpha*c_new[list_ind[i]-1]
        z_new[list_ind[i] + lag_step] = 1
        c_new[list_ind[i]+lag_step] = cmax
        
    else
        z_new[list_ind[i] + lag_step] = 0
        c_new[list_ind[i]+lag_step] = alpha*c_new[list_ind[i]+lag_step-1]
    end 
    #println(z_new)
    for item in P
        i = 1
        while i < t && y_new[item,list_ind[i]] != 1 && list_ind[i] + lag_step < t
            i += 1
        end
        #println(list_ind[i])
        if y_new[item,list_ind[i] + lag_step] == 0 
            y_new[item,list_ind[i]] = 0 
            y_new[item,(list_ind[i] + lag_step)] = 1
            
        else
            y_new[item,(list_ind[i] + lag_step)] = 0
        end 
    end 
    #display(y_new)
    
    feasible, x, I, obj = find_x_I(y_new, c_new, instance_dict)
    #println(feasible)
    if feasible
        obj += sum(set_up_cost[i,j]*y_new[i,j] for i in P, j in T) + sum(mtn_cost[j]*z_new[j] for j in T)
        sol_new = solution(x,I,y_new,z_new,c_new,obj,0)
    else
        x_I = zeros(Float64,p,t)
        sol_new = solution(x_I,x_I,y_new,z_new,c_new,unf_cost,0) 
    end 
    return sol_new
end 


In [ ]:
function mutation2(sol, instance_dict, lag_step)
    T = instance_dict["T"]
    P = instance_dict["P"]
    t = length(T)
    cmax = instance_dict["cmax"]
    alpha = instance_dict["alpha"]
    mtn_cost = instance_dict["mtn_cost"]
    demand = instance_dict["demand"]
    set_up_cost = instance_dict["set_up_cost"]
    variable_prod_cost = instance_dict["variable_prod_cost"]
    holding_cost = instance_dict["holding_cost"]
    unf_cost = sum(variable_prod_cost[i,j]*demand[i,j] + holding_cost[i,j]*demand[i,j] + set_up_cost[i,j] for i in P, j in T) + sum(mtn_cost[j] for j in T)
    #ind = rand(2:t)
    z_new = copy(sol.z)
    y_new = copy(sol.y)
    
    #println(z_new)
    #display(y_new)
    ind = rand(2:t-lag_step)
    i = 1
    while i < t && z_new[list_ind[i]] != 1 && list_ind[i] + lag_step < t
        i += 1
    end
    
    if z_new[ind] == 1 && z_new[ind + lag_step] == 0 
        z_new[ind] = 0 
        z_new[ind + lag_step] = 1
        
    elseif z_new[ind+lag_step] == 1 
        z_new[ind + lag_step] = 0
    
    else 
        z_new[ind] = 1 
        z_new[ind + lag_step] = 0
    
    end
    c_new = zeros(Float64,t)
    for j in 1:t
        if z_new[j] ==1
            c_new[j] = cmax
        else
            c_new[j] = alpha*c_new[j-1]
        end
    end
        
    
    #println(z_new)
    ind = rand(2:t-lag_step)
    for item in P
        if y_new[item, ind] == 1 && y_new[item,ind + lag_step] == 0 
            y_new[item,ind] = 0 
            y_new[item,(ind + lag_step)] = 1
            
        elseif y_new[item,(ind + lag_step)] == 1
            y_new[item,(ind + lag_step)] = 0
        
        else 
            y_new[item,ind] = 1
            y_new[item,(ind + lag_step)] = 0
        
        end 
    end 
    #display(y_new)
    
    feasible, x, I, obj = find_x_I(y_new, c_new, instance_dict)
    #println(feasible)
    if feasible
        obj += sum(set_up_cost[i,j]*y_new[i,j] for i in P, j in T) + sum(mtn_cost[j]*z_new[j] for j in T)
        sol_new = solution(x,I,y_new,z_new,c_new,obj,0)
    else
        x_I = zeros(Float64,p,t)
        sol_new = solution(x_I,x_I,y_new,z_new,c_new,unf_cost,0) 
    end 
    return sol_new
end 
    

In [36]:
pushfirst!(PyVector(pyimport("sys")."path"), "")
init  = pyimport("__init__")

PyObject <module '__init__' from '/home/harcenage/Documents/CLSPM/julia/__init__.py'>

In [4]:

        if fp 
            file = open(fp, "w") 
            write(file, 'Set of periods in the planning horizon T: \n' + str(t) + '\n' + 
                        'Set of products P: \n' + str(p) + '\n' +
                        'Capacity reduction coefficient alpha: \n' + str(alpha) + '\n' +
                        'Fixed production cost of each product in each period f: \n' + json.dumps(f) + '\n' +
                        'Variable production cost per unit of each product in each period p: \n' + json.dumps(vp) + '\n' +
                        'Inventory holding cost per unit of each product by the end of each period h: \n' +
                           json.dumps(h) + '\n' +
                         'Demand for each product in each period d: \n' + json.dumps(d) + '\n' +
                         'Maintenance cost in each period m: \n' + str(m) + '\n' +
                         'Maximum capacity of the machine cmax: \n' + str(cmax))
            close(file)
        end
        
    end 
    
end 

LoadError: syntax: character literal contains multiple characters

In [11]:
word="article"
@printf("This is an %s",word)


This is an article

In [19]:
file_path = "instances/rd_instance" + string(10)

LoadError: MethodError: no method matching +(::String, ::String)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  +([91m::PyObject[39m, ::Any) at ~/.julia/packages/PyCall/twYvK/src/pyoperators.jl:13
[0m  +(::Any, [91m::PyObject[39m) at ~/.julia/packages/PyCall/twYvK/src/pyoperators.jl:14

In [37]:
file_path = "instances/rd_instance" * string(10) * "_" * string(20) * ".txt";
instance_dict = init.gen_instance(10,20, fp=file_path); 


"instances/rd_instance10_20.txt"

Set of periods in the planning horizon T: 
20
Set of products P: 
10
Capacity reduction coefficient alpha: 
0.68
Relative decrease of capacity per set up gamma: 
0.34
Capacity reduction coefficient due to product types beta: 
[0.28, 0.29, 0.25, 0.21, 0.3, 0.29, 0.38, 0.26, 0.4, 0.27]
Fixed production cost of each product in each period f: 
[[516, 522, 737, 233, 395, 738, 582, 645, 251, 485, 742, 851, 567, 447, 650, 234, 989, 730, 671, 710], [461, 495, 617, 650, 337, 422, 228, 308, 906, 571, 718, 580, 296, 877, 499, 434, 994, 645, 310, 294], [703, 959, 379, 765, 508, 496, 636, 283, 539, 333, 871, 851, 956, 355, 476, 735, 768, 976, 902, 530], [328, 602, 676, 686, 712, 887, 896, 473, 882, 217, 430, 989, 989, 584, 350, 801, 231, 949, 232, 923], [467, 953, 651, 899, 256, 904, 553, 484, 742, 206, 319, 503, 393, 648, 201, 412, 285, 399, 256, 264], [753, 835, 633, 992, 513, 453, 499, 507, 750, 507, 572, 278, 892, 600, 974, 866, 889, 830, 873, 574], [885, 424, 601, 820, 906, 449, 348, 386, 428,

Dict{Any, Any} with 13 entries:
  "complexity"         => false
  "T"                  => 0:19
  "P"                  => 0:9
  "set_up_cost"        => [516 522 … 671 710; 461 495 … 310 294; … ; 337 463 … …
  "mtn_cost"           => [4788, 8252, 2449, 7807, 4006, 5379, 8707, 6726, 9575…
  "alpha"              => 0.68
  "demand"             => [103 318 … 453 312; 249 213 … 414 491; … ; 304 116 … …
  "beta"               => [0.28, 0.29, 0.25, 0.21, 0.3, 0.29, 0.38, 0.26, 0.4, …
  "cmax"               => 4360
  "gamma"              => 0.34
  "holding_cost"       => [18 19 … 18 10; 10 11 … 15 13; … ; 11 19 … 18 16; 16 …
  "variable_prod_cost" => [20 23 … 26 21; 23 27 … 24 24; … ; 24 22 … 22 26; 28 …
  "cyclic"             => false

In [39]:
println(instance_dict)

Dict{Any, Any}("complexity" => false, "T" => 0:9, "P" => 0:19, "set_up_cost" => [342 381 441 238 416 496 457 288 126 281; 405 298 291 387 215 247 185 186 288 108; 104 489 492 375 123 129 212 162 112 437; 376 155 477 404 321 405 493 265 193 281; 263 249 159 223 147 461 299 280 356 431; 315 103 307 264 158 469 307 413 314 115; 380 238 433 200 181 414 116 340 384 480; 313 133 146 467 291 300 138 431 432 335; 167 443 212 207 487 101 278 268 272 378; 144 419 132 282 278 247 379 248 429 486; 241 286 383 320 311 143 140 118 393 262; 337 390 291 192 157 214 260 311 171 113; 348 183 207 165 402 452 427 347 248 383; 323 357 279 483 471 455 416 278 267 292; 421 363 402 338 370 472 444 163 295 131; 209 235 178 432 185 223 212 196 346 340; 158 421 490 203 251 341 166 277 289 207; 318 432 450 468 423 418 241 310 450 460; 284 418 176 115 160 419 362 230 392 400; 410 225 316 273 231 302 358 474 454 288], "mtn_cost" => [3393, 4903, 3476, 3227, 1752, 3433, 1843, 1648, 4206, 1239], "alpha" => 0.65, "dema